# 🏠 주거취약지수 분석 및 지도 시각화

이 노트북에서는 한국의 지역별 주거취약지수를 분석하고 지도로 시각화합니다.

## 📋 분석 목표
- 자연재해 위험도와 노후주택 현황을 종합한 주거취약지수 계산
- 지역별 취약성 등급 분류 및 시각화
- 인터랙티브 지도 생성
- 분석 결과 요약 및 인사이트 도출

In [ ]:
# 라이브러리 임포트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import folium
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 한글 폰트 설정
plt.rcParams['font.family'] = 'NanumGothic'
plt.rcParams['axes.unicode_minus'] = False

print("라이브러리 임포트 완료")

## 📁 데이터 로드

In [ ]:
# 데이터 로드
data_path = '../data/processed/processed_data.csv'
df = pd.read_csv(data_path, encoding='utf-8-sig')

print(f"데이터 로드 완료: {len(df)}개 행, {len(df.columns)}개 컬럼")
print(f"컬럼명: {list(df.columns)}")
df.head()

## 🔍 데이터 탐색

In [ ]:
# 기본 정보 확인
print("=== 데이터 기본 정보 ===")
print(df.info())

print("\n=== 기술 통계 ===")
print(df.describe())

print("\n=== 결측값 현황 ===")
print(df.isnull().sum())

## 📊 주거취약지수 계산

In [ ]:
# 주거취약지수 계산
# 가중치 설정
weights = {
    'total_risk': 0.4,      # 전체 위험도 40%
    'high_risk': 0.3,       # 고위험도 30%
    'aged_housing_ratio': 0.3  # 노후주택비율 30%
}

# 주거취약지수 계산
df['housing_vulnerability_index'] = (
    df['total_risk'] * weights['total_risk'] +
    df['high_risk'] * weights['high_risk'] +
    df['aged_housing_ratio'] * weights['aged_housing_ratio']
)

# 0-100 스케일로 정규화
df['housing_vulnerability_index'] = (
    (df['housing_vulnerability_index'] - df['housing_vulnerability_index'].min()) /
    (df['housing_vulnerability_index'].max() - df['housing_vulnerability_index'].min()) * 100
)

# 등급 분류
df['vulnerability_grade'] = pd.cut(
    df['housing_vulnerability_index'],
    bins=[0, 30, 50, 70, 100],
    labels=['낮음', '보통', '높음', '매우 높음'],
    include_lowest=True
)

print("주거취약지수 계산 완료")
print(f"평균 취약지수: {df['housing_vulnerability_index'].mean():.2f}")
print(f"최고 취약지수: {df['housing_vulnerability_index'].max():.2f}")
print(f"최저 취약지수: {df['housing_vulnerability_index'].min():.2f}")

df.head()

## 📈 시각화

In [ ]:
# 시각화
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# 1. 주거취약지수 분포
axes[0, 0].hist(df['housing_vulnerability_index'], bins=20, alpha=0.7, color='skyblue')
axes[0, 0].set_title('주거취약지수 분포')
axes[0, 0].set_xlabel('취약지수')
axes[0, 0].set_ylabel('빈도')

# 2. 상위 10개 지역
top_10 = df.nlargest(10, 'housing_vulnerability_index')
axes[0, 1].barh(range(len(top_10)), top_10['housing_vulnerability_index'])
axes[0, 1].set_yticks(range(len(top_10)))
axes[0, 1].set_yticklabels(top_10['region'])
axes[0, 1].set_title('상위 10개 취약 지역')
axes[0, 1].set_xlabel('취약지수')

# 3. 위험도 vs 노후주택비율
axes[0, 2].scatter(df['total_risk'], df['aged_housing_ratio'], 
                   c=df['housing_vulnerability_index'], cmap='viridis', alpha=0.6)
axes[0, 2].set_xlabel('전체 위험도')
axes[0, 2].set_ylabel('노후주택비율 (%)')
axes[0, 2].set_title('위험도 vs 노후주택비율')
plt.colorbar(axes[0, 2].collections[0], ax=axes[0, 2])

# 4. 지역별 취약지수
axes[1, 0].bar(range(len(df)), df['housing_vulnerability_index'])
axes[1, 0].set_xticks(range(len(df)))
axes[1, 0].set_xticklabels(df['region'], rotation=45)
axes[1, 0].set_title('지역별 주거취약지수')
axes[1, 0].set_ylabel('취약지수')

# 5. 상관관계 히트맵
correlation_cols = ['total_risk', 'high_risk', 'aged_housing_ratio', 'housing_vulnerability_index']
correlation_matrix = df[correlation_cols].corr()
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', center=0, ax=axes[1, 1])
axes[1, 1].set_title('변수 간 상관관계')

# 6. 등급별 분포
grade_counts = df['vulnerability_grade'].value_counts()
axes[1, 2].pie(grade_counts.values, labels=grade_counts.index, autopct='%1.1f%%')
axes[1, 2].set_title('등급별 분포')

plt.tight_layout()
plt.show()

## 🗺️ 인터랙티브 지도 생성

In [ ]:
# 한국 지도 생성
m = folium.Map(location=[36.5, 127.5], zoom_start=7, tiles='OpenStreetMap')

# 지역별 좌표 (대략적인 중심점)
region_coords = {
    '서울특별시': [37.5665, 126.9780],
    '부산광역시': [35.1796, 129.0756],
    '대구광역시': [35.8714, 128.6014],
    '인천광역시': [37.4563, 126.7052],
    '광주광역시': [35.1595, 126.8526],
    '대전광역시': [36.3504, 127.3845],
    '울산광역시': [35.5384, 129.3114],
    '세종특별자치시': [36.4877, 127.2827],
    '경기도': [37.4138, 127.5183],
    '강원도': [37.8228, 128.1555],
    '충청북도': [36.8, 127.7],
    '충청남도': [36.5184, 126.8000],
    '전라북도': [35.7175, 127.1530],
    '전라남도': [34.8679, 126.9910],
    '경상북도': [36.4919, 128.8889],
    '경상남도': [35.4606, 128.2132],
    '제주특별자치도': [33.4996, 126.5312]
}

# 색상 매핑
def get_color(grade):
    if grade == '매우 높음':
        return 'red'
    elif grade == '높음':
        return 'orange'
    elif grade == '보통':
        return 'yellow'
    else:
        return 'green'

# 마커 추가
for idx, row in df.iterrows():
    region = row['region']
    if region in region_coords:
        lat, lon = region_coords[region]
        vulnerability = row['housing_vulnerability_index']
        grade = row['vulnerability_grade']
        color = get_color(grade)
        
        # 팝업 내용
        popup_content = f"""
        <div style='width: 200px;'>
            <h4>{region}</h4>
            <p><b>주거취약지수</b></p>
            <p>점수: {vulnerability:.1f}</p>
            <p>등급: {grade}</p>
            <p>전체 위험도: {row['total_risk']}</p>
            <p>고위험도: {row['high_risk']}</p>
            <p>노후주택비율: {row['aged_housing_ratio']:.1f}%</p>
        </div>
        """
        
        folium.CircleMarker(
            location=[lat, lon],
            radius=vulnerability/5 + 5,  # 크기 조정
            popup=folium.Popup(popup_content, max_width=300),
            color=color,
            fill=True,
            fillColor=color,
            fillOpacity=0.7
        ).add_to(m)

# 범례 추가
legend_html = """
<div style="position: fixed; 
            bottom: 50px; left: 50px; width: 150px; height: 90px; 
            background-color: white; border:2px solid grey; z-index:9999; 
            font-size:14px; padding: 10px">
            <p><b>주거취약지수 등급</b></p>
            <p><i class="fa fa-circle fa-1x" style="color:red"></i> 매우 높음</p>
            <p><i class="fa fa-circle fa-1x" style="color:orange"></i> 높음</p>
            <p><i class="fa fa-circle fa-1x" style="color:yellow"></i> 보통</p>
            <p><i class="fa fa-circle fa-1x" style="color:green"></i> 낮음</p>
</div>
"""
m.get_root().html.add_child(folium.Element(legend_html))

m

## 📊 분석 결과 요약

In [ ]:
# 분석 결과 요약
print("=== 지역별 주거취약지수 요약 ===")
summary = df[['region', 'housing_vulnerability_index', 'vulnerability_grade', 
              'total_risk', 'high_risk', 'aged_housing_ratio']].sort_values('housing_vulnerability_index', ascending=False)
print(summary)

print("\n=== 등급별 지역 수 ===")
print(df['vulnerability_grade'].value_counts())

print("\n=== 통계 요약 ===")
print(f"평균 취약지수: {df['housing_vulnerability_index'].mean():.2f}")
print(f"표준편차: {df['housing_vulnerability_index'].std():.2f}")
print(f"최고값: {df['housing_vulnerability_index'].max():.2f} ({df.loc[df['housing_vulnerability_index'].idxmax(), 'region']})")
print(f"최저값: {df['housing_vulnerability_index'].min():.2f} ({df.loc[df['housing_vulnerability_index'].idxmin(), 'region']})")

## 💾 결과 저장

In [ ]:
# 결과 저장
output_path = '../results/'
Path(output_path).mkdir(parents=True, exist_ok=True)

# CSV 파일로 저장
df.to_csv(f'{output_path}housing_vulnerability_analysis.csv', 
          index=False, encoding='utf-8-sig')

# 지도 파일로 저장
m.save(f'{output_path}housing_vulnerability_map.html')

print("=== 결과 저장 완료 ===")
print(f"- 분석 데이터: {output_path}housing_vulnerability_analysis.csv")
print(f"- 인터랙티브 지도: {output_path}housing_vulnerability_map.html")

## 📋 분석 인사이트

### 주요 발견사항
1. **주거취약지수 분포**: [분포 특성] 분포를 보이며, 평균 [평균값]점
2. **가장 취약한 지역**: [지역명] (취약지수: [점수]점)
3. **가장 안전한 지역**: [지역명] (취약지수: [점수]점)

### 정책 제언
극한 강우 대응을 위해 다음 지역들의 주거취약도 개선이 시급:
- [지역1]: [구체적 개선 방안]
- [지역2]: [구체적 개선 방안]
- [지역3]: [구체적 개선 방안]

### 추가 분석 방향
1. **시계열 분석**: 연도별 취약도 변화 추이 분석
2. **세부 지역 분석**: 시군구 단위 세분화 분석
3. **다변량 분석**: 추가 사회경제 지표와의 연관성 분석